   # Hello FLARE API

In this notebook, we go through the different commands of the FLARE API to show the syntax and usage of each.

### 1. Install NVIDIA FLARE and Provision and Start an FL System

For this notebook, we will need a running NVFLARE project that we can connect to. Follow the [Installation](https://nvflare.readthedocs.io/en/main/getting_started.html#installation) instructions to set up an environment that has NVIDIA FLARE installed if you do not have one already.

To provision and start an FL system, you can use [POC mode](setup_poc.ipynb) to quickly get started. Feel free to use an existing **provisioned** NVFLARE project if you have that available.


### 2. Connect to the FL System with the FLARE API and Get System Info

Use `new_secure_session()` to initiate a session connecting to the FL Server with the FLARE API. The necessary arguments are the username of the admin user you are using and the corresponding startup kit location.

In the code example below, we get the `admin_user_dir` by using the default project location for POC mode. Please change the workspace location and the project name to what applies for your environment if needed.

In [14]:
import os
from nvflare.fuel.flare_api.flare_api import new_secure_session
 
username = "admin@nvidia.com"
admin_user_dir = os.path.join("/tmp/nvflare/poc/example_project/prod_00", username)
sess = new_secure_session(
    username=username,
    startup_kit_location=admin_user_dir
)
print(sess.get_system_info())

SystemInfo
server_info:
status: stopped, start_time: Wed Aug 30 19:15:17 2023
client_info:
site-3(last_connect_time: Wed Aug 30 19:16:08 2023)
site-1(last_connect_time: Wed Aug 30 19:16:09 2023)
site-2(last_connect_time: Wed Aug 30 19:16:09 2023)
job_info:



Note that if debug mode is not enabled, there is no output after initiating a session successfully, so we print the output of `get_system_info()`.

The `get_system_info()` command does not take any arguments and returns a SystemInfo object consisting of server_info (server status and start time), client_info (each connected client and the last connect time for that client), and job_info (the list of current jobs with the job_id and app_name).

If you are unable to connect and initiate a session, make sure that your FL Server is running and that the configurations are correct with the right path to the admin startup kit directory.

### 3. Prepare Examples

We are going to use hello-numpy-sag for an example job to submit. The following command copies the example to the place jobs are expected assuming you are using the default poc workspace.

In [10]:
! cp -r  ../hello-world/hello-numpy-sag /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/transfer/.

In [11]:
! ls -al /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/transfer/

total 100
drwxr-xr-x  7 chester chester  4096 Aug 30 19:14 .
drwxr-xr-x 21 chester chester  4096 Aug 30 17:49 ..
drwxr-xr-x 18 chester chester  4096 Aug 30 15:58 advanced
-rw-rw-r--  1 chester chester   210 Mar 13 11:35 .gitignore
drwxr-xr-x  3 chester chester  4096 Aug 30 19:14 hello-numpy-sag
drwxr-xr-x 10 chester chester  4096 Aug 30 17:49 hello-world
drwxrwxr-x  2 chester chester  4096 Jul 26 21:44 .ipynb_checkpoints
-rw-rw-r--  1 chester chester 19506 Mar 15 21:01 jupyterlab_kernel.png
-rw-rw-r--  1 chester chester 19544 Aug 30 15:58 nvflare_getting_started.ipynb
-rw-rw-r--  1 chester chester  3599 Aug 30 15:58 nvflare_setup.ipynb
-rw-rw-r--  1 chester chester 19877 Aug 30 15:58 README.md
-rwxrwxr-x  1 chester chester   125 Mar 15 21:01 set_env.sh
drwxr-xr-x  3 chester chester  4096 Aug 30 19:14 tutorials


### 4. Submit Job
With a session successfully connected, you can use `submit_job()` to submit your job. You can change `path_to_example_job` to the location of the job you are submitting (here, we are using the example job we copied to our transfer directory in the previous step). Upon successful submission, the job_id is returned as a string.

In [12]:
path_to_example_job = "hello-numpy-sag/jobs/hello-numpy-sag"
job_id = sess.submit_job(path_to_example_job)
print(job_id + " was submitted")

f484fcb5-24cd-47d7-badd-bcfbfde9e73a was submitted


### 5. Monitor Job

The command `monitor_job()` allows you to follow a job until the job is done.

By default, `monitor_job()` only has one required arguement, the `job_id` of the job you are waiting for, and the default behavior is to wait until the job is complete before returning a Return Code of `JOB_FINISHED`.

In order to follow along and see a more meaningful result and demonstrate some of the possibilities of using this function, the following cell contains a `sample_cb()` callback that keeps track of the number of times the callback is run and prints the `job_meta` the first three times and the final time before `monitor_job()` completes with every other call just printing a dot to save output space. This callback is just an example of what can be done with additional arguments and the `job_meta` information of the job that is being monitored. You can use logic to return a value other than True to conditionally stop `monitor_job()` and return `MonitorReturnCode.ENDED_BY_CB`.

In [13]:
from nvflare.fuel.flare_api.flare_api import Session

def sample_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3:
            print(job_meta)
            print(cb_kwargs["cb_run_counter"])
        else:
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True

sess.monitor_job(job_id, cb=sample_cb, cb_run_counter={"count":0})

{'name': 'hello-numpy-sag', 'resource_spec': {}, 'min_clients': 2, 'deploy_map': {'app': ['@ALL']}, 'job_folder_name': 'hello-numpy-sag', 'submitter_name': 'admin@nvidia.com', 'submitter_org': 'nvidia', 'submitter_role': 'project_admin', 'job_id': 'f484fcb5-24cd-47d7-badd-bcfbfde9e73a', 'submit_time': 1693448069.014861, 'submit_time_iso': '2023-08-30T19:14:29.014861-07:00', 'start_time': '2023-08-30 19:14:29.361154', 'duration': 'N/A', 'status': 'RUNNING', 'job_deploy_detail': ['server: OK', 'site-3: OK', 'site-1: OK', 'site-2: OK'], 'schedule_count': 1, 'last_schedule_time': 1693448069.293098, 'schedule_history': ['2023-08-30 19:14:29: scheduled']}
{'count': 0}
{'name': 'hello-numpy-sag', 'resource_spec': {}, 'min_clients': 2, 'deploy_map': {'app': ['@ALL']}, 'job_folder_name': 'hello-numpy-sag', 'submitter_name': 'admin@nvidia.com', 'submitter_org': 'nvidia', 'submitter_role': 'project_admin', 'job_id': 'f484fcb5-24cd-47d7-badd-bcfbfde9e73a', 'submit_time': 1693448069.014861, 'submit

<MonitorReturnCode.JOB_FINISHED: 0>

### 6. Get Job Meta

To get the job meta information, you can use the `get_job_meta()` command. The only argument required for this command is the job id for the job you are getting the meta for. The job meta information will be returned as an object.

In [ ]:
sess.get_job_meta(job_id)

### 7. List Jobs

To get the information for what jobs have been submitted to the server, you can use the `list_jobs()` command. If no arguments are included, the defaults are False for both "detailed" and "reverse".

Setting "detailed" to True will return more detailed information about each job.

Setting "reverse" to True will return the jobs in reverse order by submission time.

The arg "limit" can be set to specify the maximum number of jobs to return, with 0 or None meaning return all jobs (the default is None to show all).

The args "id_prefix" and "name_prefix" can be used to further filter the jobs returned to have an id or name beginning with the string set for the respective argument.

In [ ]:
import json

def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))

list_jobs_output = sess.list_jobs()
print(format_json(list_jobs_output))


In [ ]:

list_jobs_output_detailed = sess.list_jobs(detailed=True)
print(format_json(list_jobs_output_detailed))

### 8. Download Job Result

The `download_job_result()` command downloads the job result to the "download_dir" (this is usually set in fed_admin.json in the startup directory of the admin_user_dir used when launching the FLARE API Session, and this value is relative to the admin_user_dir). This command only has one required arguement, the `job_id` of the job result to download.

In [ ]:
sess.download_job_result(job_id)

### 9. Clone Jobs

To clone an existing job, you can use the `clone_job()` command. The `clone_job()` command only has one required arguement, the `job_id` of the job to clone.

In [ ]:
sess.clone_job(job_id)

### 10. Abort Job

If training gets stuck or there is another reason to stop a running job, you can use the `abort_job()` command. The `abort_job()` command only has one required arguement, the `job_id` of the job to abort, and None is returned upon successfully issuing the command.

In [ ]:
print(job_id)
sess.abort_job(job_id)

### 11. Delete Job

You should be able to see the output in the terminals where you are running your FL Server and Clients when you submitted the job. You can also use `monitor_job()` to follow along and give you updates on the progress until the job is done.

By default, `delete_job()` only has one required argument, the `job_id` of the job you are waiting for, and the default behavior is to return nothing if the command is successful. The command will raise a `JobNotDone` exception if the job is still running. 

In [ ]:
sess.delete_job(job_id)

### 12. All Other Commands

For all other commands that do not have a specific FLARE API implementation, using ``api.do_command()`` should provide what the FLAdminAPI would have returned in the raw result.

In [ ]:
sess.api.do_command("list_sp")